## Strategy backtesting notebook
The following notebook is used to backtest the strategy and generate results to be analyzed later. 

How the data looks:
"17-09":{
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "6-month":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "12-month":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "max":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   }
}

### Env variables and imports

In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import os.path
import sys
import uuid
import copy
import json
# import warnings

import pandas as pd
import requests
import tqdm
import backtrader as bt
import pyfolio as pf
from matplotlib import warnings
warnings.filterwarnings("ignore")  # Avoid some noise

In [10]:
import datetime 
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import numpy as np

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = 'weekly'

# FRAMEWORK = 'dask'
FRAMEWORK = 'pandas'

# PERC_CHANGE_METHOD = 'mean'
PERC_CHANGE_METHOD = 'order aware'

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/forward_opt_results.json'
FORWARD_OPT_RESULTS_JSON = 'test.json'
BACKWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/backward_opt_results.json'
BACKWARD_OPT_RESULTS_JSON = 'test_b.json'
# TODO: use proper json path



In [11]:
# TODO: remove the mdo and pdo indicators as well as 64, 132, 256, 512 periods for simplicity
metrics_options = ["adx", "ppo", "stochastic"]
size_options = [8, 16, 32]

In [12]:
# TODO: is this needed?
keys = [end_date.strftime('%y-%m') for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE)]

timeframes = [
    'weekly',
#     'daily' # TODO
]
range_values = [
    '6-month',
    '12-month',
    'max',
]
forward_func_values = [
    'mean',
    'order'
]
backward_func_values = [
    'mean',
#     None, # TODO
#     'order', # TODO
#     'linear', # TODO
#     'kNN', # TODO
]
backward_func_options = [
    'single',
#     'double' # TODO
]
perc_allocated_per_trade = [
    3,
    5,
    10,
    20,
    25
]

In [15]:
from itertools import product

all_strategy_combinations = list(product(timeframes, range_values, forward_func_values, backward_func_options, backward_func_values, perc_allocated_per_trade))
all_strategy_combinations

[('weekly', '6-month', 'mean', 'single', 'mean', 3),
 ('weekly', '6-month', 'mean', 'single', 'mean', 5),
 ('weekly', '6-month', 'mean', 'single', 'mean', 10),
 ('weekly', '6-month', 'mean', 'single', 'mean', 20),
 ('weekly', '6-month', 'mean', 'single', 'mean', 25),
 ('weekly', '6-month', 'order', 'single', 'mean', 3),
 ('weekly', '6-month', 'order', 'single', 'mean', 5),
 ('weekly', '6-month', 'order', 'single', 'mean', 10),
 ('weekly', '6-month', 'order', 'single', 'mean', 20),
 ('weekly', '6-month', 'order', 'single', 'mean', 25),
 ('weekly', '12-month', 'mean', 'single', 'mean', 3),
 ('weekly', '12-month', 'mean', 'single', 'mean', 5),
 ('weekly', '12-month', 'mean', 'single', 'mean', 10),
 ('weekly', '12-month', 'mean', 'single', 'mean', 20),
 ('weekly', '12-month', 'mean', 'single', 'mean', 25),
 ('weekly', '12-month', 'order', 'single', 'mean', 3),
 ('weekly', '12-month', 'order', 'single', 'mean', 5),
 ('weekly', '12-month', 'order', 'single', 'mean', 10),
 ('weekly', '12-mont

In [16]:
with open('parsed_combinations.json', 'r') as f:
    parsed_combinations = json.loads(json.dumps(all_strategy_combinations)) # json.loads(f.read())

unparsed_strategy_combinations = [entry for entry in all_strategy_combinations if tuple(entry) not in parsed_combinations]
unparsed_strategy_combinations

[('weekly', '6-month', 'mean', 'single', 'mean', 3),
 ('weekly', '6-month', 'mean', 'single', 'mean', 5),
 ('weekly', '6-month', 'mean', 'single', 'mean', 10),
 ('weekly', '6-month', 'mean', 'single', 'mean', 20),
 ('weekly', '6-month', 'mean', 'single', 'mean', 25),
 ('weekly', '6-month', 'order', 'single', 'mean', 3),
 ('weekly', '6-month', 'order', 'single', 'mean', 5),
 ('weekly', '6-month', 'order', 'single', 'mean', 10),
 ('weekly', '6-month', 'order', 'single', 'mean', 20),
 ('weekly', '6-month', 'order', 'single', 'mean', 25),
 ('weekly', '12-month', 'mean', 'single', 'mean', 3),
 ('weekly', '12-month', 'mean', 'single', 'mean', 5),
 ('weekly', '12-month', 'mean', 'single', 'mean', 10),
 ('weekly', '12-month', 'mean', 'single', 'mean', 20),
 ('weekly', '12-month', 'mean', 'single', 'mean', 25),
 ('weekly', '12-month', 'order', 'single', 'mean', 3),
 ('weekly', '12-month', 'order', 'single', 'mean', 5),
 ('weekly', '12-month', 'order', 'single', 'mean', 10),
 ('weekly', '12-mont

In [18]:
timeframe, period, forward_opt_func, backward_opt_func, backward_opt_func_option, perc_allocated = unparsed_strategy_combinations[0]
timeframe, period, forward_opt_func, backward_opt_func, backward_opt_func_option, perc_allocated

('weekly', '6-month', 'mean', 'single', 'mean', 3)

### Utilities

In [ ]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        return json.loads(f.read())

FORWARD_OPT_JSON = get_forward_opt_json()

def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, 'r') as f:
        return json.loads(f.read())

BACKWARD_OPT_JSON = get_backward_opt_json()   


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False
    
    if range_val not in BACKWARD_OPT_JSON[key]:
        return False
    
    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False
    
    return True

In [20]:
def get_ticker_csv_path(ticker_name, timeframe):
    return f"/home/narboom23/Projects/licenta/data/{timeframe}_tickers/{ticker_name}.csv"


# def get_ticker_csv_as_df(ticker_name, timeframe):
#     return pd.read_csv(get_ticker_csv_path(ticker_name, timeframe))

### Strategy
The main strategy class used

In [19]:
class RaynerTeoStrategy(bt.Strategy):
    params = (
        ('timeframe', 'daily'),
        ('period', 'max'),
        ('forward_opt_func', 'mean'),
        ('backward_opt_func', 'mean'),
        ('backward_opt_func_option', 'single'),
    )

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()

            self.inds[d]["sma180"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["sma200"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["sma220"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["sma240"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["sma260"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["sma280"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            
            self.inds[d]["rsi2"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi4"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi6"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi8"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi10"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi12"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )
            self.inds[d]["rsi14"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )

            self.inds[d]["adx8"] = bt.indicators.ADX(d, period=8)
            self.inds[d]["adx16"] = bt.indicators.ADX(d, period=16)
            self.inds[d]["adx32"] = bt.indicators.ADX(d, period=32)

            self.inds[d]["ppo8"] = bt.indicators.PPO(
                d.close, period1=8, period2=self.params.maperiod
            )  # , period_signal=?)
            self.inds[d]["ppo16"] = bt.indicators.PPO(
                d.close, period1=16, period2=self.params.maperiod
            )  # , period_signal=?)
            self.inds[d]["ppo32"] = bt.indicators.PPO(
                d.close, period1=32, period2=self.params.maperiod
            )  # , period_signal=?)
            
            self.inds[d]["stochastic8"] = bt.indicators.Stochastic(
                d, period=8, safediv=True
            )
            self.inds[d]["stochastic16"] = bt.indicators.Stochastic(
                d, period=16, safediv=True
            )
            self.inds[d]["stochastic32"] = bt.indicators.Stochastic(
                d, period=32, safediv=True
            )

            self.inds[d]["order_placed_days_ago"] = 0
        
    def next(self):
        optimal_params = get_forward_optimal_params(date, self.params.period) # TODO
        optimal_sma = optimal_params['sma']
        optimal_rsi_open = optimal_params['rsi_open_period']
        optimal_days_ago = optimal_params['days_ago']
        
        backward_optimal_params = get_backward_optimal_params(date, self.params.period)  # TODO

        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size

            if not pos:
                if d.close[0] > self.inds[d][optimal_sma][0] and self.inds[d][optimal_rsi_open][0] < 30:
                    self.buy(data=d)
            else:
                if self.params.backward_opt_func == None: 
                    if self.inds[d][optimal_rsi_open][0] >= self.params.rsi_close_period or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago:
                        self.sell(data=d)
                        self.inds[d]["order_placed_days_ago"] = 0
                    else:
                        self.inds[d]["order_placed_days_ago"] += 1
                else:
                    barrier = backward_optimal_params[self.params.backward_opt_func_option]['barrier']
                    indicator = backward_optimal_params[self.params.backward_opt_func_option]['indicator']
                    
                    if self.params.backward_opt_func_option == 'lower':
                        new_condition = self.inds[d][indicator] <= barrier
                    else:
                        new_condition = self.inds[d][indicator] >= barrier
                        
                    if new_condition is True:
                        if self.inds[d][optimal_rsi_open][0] >= self.params.rsi_close_period or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago:
                            self.sell(data=d)
                            self.inds[d]["order_placed_days_ago"] = 0
                        else:
                            self.inds[d]["order_placed_days_ago"] += 1
                    else:
                        self.inds[d]["order_placed_days_ago"] += 1

    def stop(self):
        self.log(
            f"(timeframe {self.params.timeframe}, "
            f"period {self.params.period}, "
            f"forward opt func {self.params.forward_opt_func}, "
            f"backward opt function {self.params.backward_opt_func}"
            f"backward opt function option {self.params.backward_opt_func_option}) "
            f"Ending Value {self.broker.getvalue()}",
            doprint=True,
        )

## Run configuration

In [ ]:
DEFAULT_FROM_DATE = datetime.datetime(2004, 1, 1)
DEFAULT_TO_DATE = datetime.datetime(2020, 12, 31)
DEFAULT_CASH = 100.0
DEFAULT_COMMISION = 0.0
DEFAULT_CPU_COUNT = 4
DEFAULT_TICKER_LIST = []

In [ ]:
perc_allocated_per_trade = [
    3,
    5,
    10,
    20,
    25
]

timeframes = [
    'weekly',
#     'daily' # TODO
]

range_values = [
    '6-month',
    '12-month',
    'max',
]
forward_func_values = [
    'mean',
    'order'
]
backward_func_values = [
    'mean',
#     None, # TODO
#     'order', # TODO
#     'linear', # TODO
#     'kNN', # TODO
]
backward_func_options = [
    'lower',
    'upper'
    # TODO
    #     'single',
    #     'double'
]

In [21]:
list(product(timeframes, perc_allocated_per_trade))

[('weekly', 3), ('weekly', 5), ('weekly', 10), ('weekly', 20), ('weekly', 25)]

In [ ]:
PERC_ALLOCATED = 25
TIMEFRAME = 'weekly'

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)


# Add a strategy
# strats = cerebro.addstrategy(RaynerTeoStrategy, **{
strats = cerebro.optstrategy(RaynerTeoStrategy, **{
    'timeframe': TIMEFRAME,
    'period': range_values,
    'forward_opt_func': forward_func_values,
    'backward_opt_func': backward_func_values,
    'backward_opt_func_option': backward_func_options,
})


datalist = [
    (get_ticker_csv_path(ticker_name, TIMEFRAME), ticker_name) for ticker_name in DEFAULT_TICKER_LIST
]

for i in range(len(DEFAULT_TICKER_LIST)):
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datalist[i][0],
        name=datalist[i][1],
        # Do not pass values before this date
        fromdate=DEFAULT_FROM_DATE,
        # Do not pass values before this date
        todate=DEFAULT_TO_DATE,
        # Do not pass values after this date
        reverse=False,
    )
    data.plotinfo.plot = False

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(DEFAULT_CASH)

# Add pyfolio analyzer for stats
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.PercentSizer, percents=PERC_ALLOCATED)

# Set the commission
cerebro.broker.setcommission(commission=DEFAULT_COMMISION)

# Run over everything
strat = cerebro.run(maxcpus=DEFAULT_CPU_COUNT)

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
pyfoliozer = strat[1][0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

In [ ]:
strat[0][0]

In [ ]:
dir(strat[0][0])

In [ ]:
dir(strat[0][0].params)

In [ ]:
strat[0][0].params.maperiod

In [ ]:
strat[1][0].params.maperiod

In [ ]:
# TODO: find function and get this as a dataframe
# TODO: find a way to also save this so that you can display the result later
pf.create_simple_tear_sheet(returns)

In [ ]:
run strategy optimization
save results to file

In [ ]:
parse results
find best by field
